In [1]:
import random
import pandas as pd
import random_name
from sklearn.datasets import make_classification
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from tqdm import tqdm_notebook

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

C:\Users\victolee\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\victolee\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\victolee\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\victolee\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## 데이터 수 관련 파라미터

In [2]:
NUM_SUB_DATASETS = 32
NUM_SAMPLES = 2048
NUM_FEATURES = 255
MAX_SEED = 2**32 - 1

In [3]:
def create_dataset(random_seed):
    random.seed(3 + random_seed
    X,y = make_classification(n_samples=NUM_SAMPLES, 
                             n_features=NUM_FEATURES, 
                             n_informative=random.randint(33,47),
                             n_redundant=0,
                             n_repeated=0,
                             n_classes=2,
                             n_clusters_per_class=3,
                             weights=None,
                             flip_y=0.05,
                             class_sep=1.0,
                             hypercube=True,
                             shift=0.0,
                             scale=1.0,
                             shuffle=True,
                             random_state=random_seed)
    df = pd.DataFrame(X, columns=['V' + str(x) for x in range(0,len(X[0]))])
    df['magic'] = random_seed
    df['target'] = y
    return df

In [4]:
seed_list = random.sample(range(1,MAX_SEED), NUM_SUB_DATASETS)
df = pd.concat([create_dataset(s) for s in seed_list], axis=0, sort = False).reset_index(drop=True)
df = df.sample(frac=1, random_state=9726).reset_index(drop=True)

le = preprocessing.LabelEncoder()
le.fit(df['magic'])
df['magic'] = le.transform(df['magic']) 

In [5]:
train = df.groupby(['magic']).sample(frac=0.75, random_state=17)
test = df.drop(train.index, axis=0)

In [7]:
train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)


In [ ]:
public_test = test.groupby(['magic']).sample(frac=0.5, random_state=17)
private_test = test.drop(public_test.index, axis=0)
public_test = public_test.sample(frac=1).reset_index(drop=True)
private_test = private_test.sample(frac=1).reset_index(drop=True)

### 모델링 결과 (코드 출처 : https://www.kaggle.com/speedwagon/quadratic-discriminant-analysis

In [11]:
cols = [c for c in train.columns if c not in ['id', 'target', 'magic']]

oof = np.zeros(len(train))
preds = np.zeros(len(test))

for i in tqdm_notebook(range(NUM_SUB_DATASETS)):

    train2 = train[train['magic']==i]
    test2 = test[test['magic']==i]
    idx1 = train2.index; idx2 = test2.index
    train2.reset_index(drop=True,inplace=True)

    data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
    data2 = VarianceThreshold(threshold=2).fit_transform(data[cols])

    train3 = data2[:train2.shape[0]]; test3 = data2[train2.shape[0]:]

    skf = StratifiedKFold(n_splits=11, shuffle=True, random_state=42)
    for train_index, test_index in skf.split(train2, train2['target']):

        clf = QuadraticDiscriminantAnalysis()
        clf.fit(train3[train_index,:],train2.loc[train_index]['target'])
        oof[idx1[test_index]] = clf.predict_proba(train3[test_index,:])[:,1]
        preds[idx2] += clf.predict_proba(test3)[:,1] / skf.n_splits

auc = roc_auc_score(train['target'], oof)
print(f'AUC: {auc:.5}')

C:\Users\victolee\AppData\Local\Temp/ipykernel_3396/4089557962.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(NUM_SUB_DATASETS)):


  0%|          | 0/32 [00:00<?, ?it/s]

AUC: 0.9709


In [13]:
train['target'].value_counts()

1    16485
0    16283
Name: target, dtype: int64

In [12]:
train.to_csv("train.csv", encoding="utf-8", index=False)

In [24]:
test.to_csv("test.csv", encoding="utf-8", index=False)

In [19]:
sample_submission = pd.DataFrame([test['target']]).T

In [17]:
sample_submission = pd.DataFrame([range(test['target'].shape[0]),test['target']]).T

In [21]:
sample_submission.to_csv('answer.csv')

In [22]:
sample_submission['target'] = 1
sample_submission

,target
0,1
1,1
2,1
3,1
4,1
...,...
16379,1
16380,1
16381,1
16382,1


In [23]:
sample_submission.to_csv('sample_submission.csv')

In [14]:

public_test.to_csv("test.csv", encoding="utf-8", index=False)
private_test.to_csv("private.csv", encoding="utf-8", index=False)